In [1]:
import pandas as pd
import numpy as np
import re
from string import ascii_letters, digits
import utils.cleaning_dicts
import os
import sys

import warnings
warnings.filterwarnings('ignore')

print(utils.cleaning_dicts.clean_team_pff_full)

os.chdir("/home/tomb/nfl_models/")

!ls

{'arizona-cardinals': 'ari', 'atlanta-falcons': 'atl', 'baltimore-ravens': 'bal', 'buffalo-bills': 'buf', 'carolina-panthers': 'car', 'chicago-bears': 'chi', 'cincinnati-bengals': 'cin', 'cleveland-browns': 'cle', 'dallas-cowboys': 'dal', 'denver-broncos': 'den', 'detroit-lions': 'det', 'green-bay-packers': 'gb', 'houston-texans': 'hou', 'indianapolis-colts': 'ind', 'jacksonville-jaguars': 'jax', 'kansas-city-chiefs': 'kc', 'los-angeles-chargers': 'lac', 'los-angeles-rams': 'lar', 'miami-dolphins': 'mia', 'minnesota-vikings': 'min', 'new-england-patriots': 'ne', 'new-orleans-saints': 'no', 'new-york-giants': 'nyg', 'new-york-jets': 'nyj', 'oakland-raiders': 'lv', 'philadelphia-eagles': 'phi', 'pittsburgh-steelers': 'pit', 'san-francisco-49ers': 'sf', 'seattle-seahawks': 'sea', 'tampa-bay-buccaneers': 'tb', 'tennessee-titans': 'ten', 'washington-redskins': 'was', 'washington-football-team': 'was', 'las-vegas-raiders': 'lv', 'washington-commanders': 'was', 'washingtoncommanders': 'was', 

In [5]:
cur_week_int = 4
cur_week_str = str(4)

### read in weather data###
weather = pd.read_csv('./current_data/week_4/weather_hist_all.csv')

def raiders(df):
    if 'oak' in str(df.away_matchup_id) and '2020' in str(df.away_matchup_id):
        return df.away_matchup_id.replace("oak","lv")
    if 'oak' in str(df.away_matchup_id) and '2021' in str(df.away_matchup_id):
        return df.away_matchup_id.replace("oak","lv")
    if 'oak' in str(df.away_matchup_id) and '2022' in str(df.away_matchup_id):
        return df.away_matchup_id.replace("oak","lv")
    else:
        return df.away_matchup_id
weather['away_matchup_id'] = weather.apply(lambda df: raiders(df), axis=1)


In [6]:

oak = {'oak':'lv'}  

spreads = pd.read_csv("./current_data/week_4/spreads_w"+cur_week_str+".csv")

spreads['team_home_abb'] = spreads['team_home_abb'].map(oak).fillna(spreads['team_home_abb'])
spreads['away_team_abb'] = spreads['away_team_abb'].map(oak).fillna(spreads['away_team_abb'])  
spreads = spreads[spreads['schedule_season']>=2014]
spreads = spreads[['schedule_season','schedule_week','team_home_abb','score_home','score_away','away_team_abb','team_favorite_id','spread_favorite','over_under_line','starting_spread', 'Total Score Open',
       'fav_team_open', 'fav_team_cur', 'remain_fav', 'spread_movement','ou_movement', 'strong_movement', 'fav_team_stronger']]
spreads['team_home_abb'] = spreads['team_home_abb'].astype(str)
spreads['team_favorite_id'] = spreads['team_favorite_id'].astype(str)
spreads['over_under_line'] = spreads['over_under_line'].astype(float)


def fav_spread(nData):
    if nData['team_home_abb'] == nData['team_favorite_id']:
        return nData['spread_favorite']
    elif nData['away_team_abb'] == nData['team_favorite_id']:
        return nData['spread_favorite']
    else:
        pass
spreads['fav_spread'] = spreads.apply(lambda nData: fav_spread(nData), axis=1)

def nonfav_spread(nData):
    if nData['team_home_abb'] != nData['team_favorite_id']:
        return nData['team_home_abb']
    elif nData['away_team_abb'] != nData['team_favorite_id']:
        return nData['away_team_abb']
    else:
        pass
spreads['team_notfav_id'] = spreads.apply(lambda nData: nonfav_spread(nData), axis=1)

def cover_or_not(nData):    
    if nData['team_home_abb'] == nData['team_favorite_id']:
        if ((nData['score_home']-nData['score_away']))+nData['spread_favorite'] > 0:
            return 'Cover'
        elif ((nData['score_home']-nData['score_away']))+nData['spread_favorite'] == 0:            
            return 'Push'       
        else:            
            return 'No Cover'
    elif nData['away_team_abb'] == nData['team_favorite_id']:        
        if ((nData['score_away']-nData['score_home']))+nData['spread_favorite'] > 0:            
            return 'Cover'        
        elif ((nData['score_away']-nData['score_home']))+nData['spread_favorite'] == 0:            
            return 'Push'        
        else:            
            return 'No Cover'
spreads['fav_cover'] = spreads.apply(lambda nData: cover_or_not(nData), axis=1)

def OU_or_not(nData):    
    if (nData['score_home']+nData['score_away']) > nData['over_under_line']:        
        return 'Over'    
    elif (nData['score_home']-nData['score_away']) == nData['over_under_line']:        
        return 'Push'    
    else:        
        return 'Under'
spreads['over_under_result'] = spreads.apply(lambda nData: OU_or_not(nData), axis=1)



spreads['schedule_season'] = spreads['schedule_season'].apply(int)    
spreads['schedule_week'] = spreads['schedule_week'].apply(int)  
data = spreads.sort_values(by=["team_home_abb","schedule_season","schedule_week"], ascending=[True, True, True])

def clean_spreads(df):
    ##  basic scrubbing to clean data ##    
    df['schedule_season'] = df['schedule_season'].apply(str)    
    df['schedule_week'] = df['schedule_week'].apply(str)        
    df=df.apply(lambda x: x.astype(str).str.lower())    
    #df['schedule_week']=df['schedule_week'].astype(str).str[:-2].astype(object)    
    #df['schedule_season'] = df['schedule_season'].astype(str).str[:-2].astype(object)        
    
    ##  create our unique ids  ##
    df.insert(0, "home_matchup_id", (df['team_home_abb']+'vs'+df['away_team_abb']+'_'+df['schedule_season']+'_'+df['schedule_week']))
    df.insert(1, "away_matchup_id", (df['away_team_abb']+'@'+df['team_home_abb']+'_'+df['schedule_season']+'_'+df['schedule_week']))
    df.insert(2, "home_id", (df['team_home_abb']+'_'+df['schedule_season']+'_'+df['schedule_week']))
    df.insert(3, "away_id", (df['away_team_abb']+'_'+df['schedule_season']+'_'+df['schedule_week']))
    return df
    
data = clean_spreads(data)

data = pd.merge(data, weather, on='away_matchup_id', how='left')


sh = data
sa = data

sh = sh.rename(columns={'home_id':'team_id'})
sh.drop('away_id', axis=1, inplace=True)

sa = sa.rename(columns={'away_id':'team_id'})
sa.drop('home_id', axis=1, inplace=True)

spread_comb = pd.concat([sh, sa], axis=0)
spread_comb['team_abb'] = spread_comb['team_id'].astype(str).str[:3]
spread_comb['team_abb'] = spread_comb['team_abb'].str.replace("_","")

def hora1(nData):
    if nData['team_favorite_id'] == nData['team_home_abb']:
        return 1
    elif nData['team_notfav_id'] == nData['team_home_abb']:
        return 1
    else:
        return 0
spread_comb['homeoraway'] = spread_comb.apply(lambda nData: hora1(nData), axis=1)

def hora(nData):
    if nData['team_favorite_id'] == nData['away_team_abb']:
        return 1
    else:
        return 0
spread_comb['fav_homeoraway'] = spread_comb.apply(lambda nData: hora(nData), axis=1)
#sh['fav_homeoraway'] = sh.apply(lambda nData: hora(nData), axis=1)

def ws(nData):
    if (nData['fav_homeoraway'] == 0) & (nData['fav_cover'] == 'cover'):
        return 1
    elif (nData['fav_homeoraway'] == 1) & (nData['fav_cover'] == 'no cover'):
        return 1
    else:
        return 0

def ls(nData):    
    if (nData['fav_homeoraway'] == 0) & (nData['fav_cover'] == 'no cover'):
        return 1
    elif (nData['fav_homeoraway'] == 1) & (nData['fav_cover'] == 'cover'):
        return 1
    else:
        return 0

spread_comb['ats_w'] = spread_comb.apply(lambda nData: ws(nData), axis=1)
spread_comb['ats_l'] = spread_comb.apply(lambda nData: ls(nData), axis=1)
spread_comb.tail(n=15).T



,2113,2114,2115,2116,2117,2118,2119,2120,2121,2122,2123,2124,2125,2126,2127
home_matchup_id,wasvsdal_2020_7,wasvsnyg_2020_9,wasvscin_2020_11,wasvssea_2020_15,wasvscar_2020_16,wasvslac_2021_1,wasvsnyg_2021_2,wasvsno_2021_5,wasvskc_2021_6,wasvstb_2021_10,wasvssea_2021_12,wasvsdal_2021_14,wasvsphi_2021_17,wasvsjax_2022_1,wasvsphi_2022_3
away_matchup_id,dal@was_2020_7,nyg@was_2020_9,cin@was_2020_11,sea@was_2020_15,car@was_2020_16,lac@was_2021_1,nyg@was_2021_2,no@was_2021_5,kc@was_2021_6,tb@was_2021_10,sea@was_2021_12,dal@was_2021_14,phi@was_2021_17,jax@was_2022_1,phi@was_2022_3
team_id,dal_2020_7,nyg_2020_9,cin_2020_11,sea_2020_15,car_2020_16,lac_2021_1,nyg_2021_2,no_2021_5,kc_2021_6,tb_2021_10,sea_2021_12,dal_2021_14,phi_2021_17,jax_2022_1,phi_2022_3
schedule_season,2020,2020,2020,2020,2020,2021,2021,2021,2021,2021,2021,2021,2021,2022,2022
schedule_week,7,9,11,15,16,1,2,5,6,10,12,14,17,1,3
team_home_abb,was,was,was,was,was,was,was,was,was,was,was,was,was,was,was
score_home,25.0,20.0,20.0,15.0,13.0,16.0,30.0,22.0,13.0,29.0,17.0,20.0,16.0,28.0,8.0
score_away,3.0,23.0,9.0,20.0,20.0,20.0,29.0,33.0,31.0,19.0,15.0,27.0,20.0,22.0,24.0
away_team_abb,dal,nyg,cin,sea,car,lac,nyg,no,kc,tb,sea,dal,phi,jax,phi
team_favorite_id,was,was,was,sea,car,lac,was,no,kc,tb,sea,dal,phi,was,phi


In [7]:
spread_comb.away_team_abb.value_counts()

min    134
gb     134
chi    134
jax    134
sf     134
buf    134
tb     134
ne     134
atl    134
no     134
sea    134
kc     134
ari    134
lar    134
phi    134
was    134
ind    132
hou    132
car    132
mia    132
cle    132
pit    132
lac    132
dal    132
den    132
nyg    132
ten    132
nyj    132
cin    132
bal    132
det    132
lav     96
lv      36
Name: away_team_abb, dtype: int64

In [8]:
spread_comb.team_home_abb.value_counts()

lac    134
dal    134
nyg    134
nyj    134
pit    134
ind    134
hou    134
den    134
det    134
cle    134
cin    134
car    134
ten    134
bal    134
mia    134
ne     132
sf     132
tb     132
sea    132
phi    132
no     132
ari    132
min    132
lar    132
atl    132
kc     132
jax    132
gb     132
chi    132
buf    132
was    132
lav     96
lv      38
Name: team_home_abb, dtype: int64

In [9]:
def raiders_fo(df):
    if 'oak' in str(df.team) and '2020' in str(df.year):
        return df.team_id.str.replace("oak_","lv_")
    elif 'oak' in str(df.team) and '2021' in str(df.year):
        return df.team_id.str.replace("oak_","lv_")
    else:
        return str(df.team_id)

def rolling_fo(data=None, roll_value=None, roll_type=None):
    
    """
        Args:
        data: input pandas dataframe to be rolled
        roll_value: input the number, default is three ## we will need to modify the function if we want more ##
        roll_type: 'mean','std', or 'var' are the only options at the point
        ## assign mean for a given team & year as opposed to the entire dataset
   
    """
    
    data = data.sort_values(by=["team","year","week"], ascending=[True, True, True])
    #data=data.fillna(data.mean())
    num_cols = ['total_dvoa', 'off_dvoa','off_pass_dvoa', 'off_rush_dvoa', 'def_dvoa', 'def_pass_dvoa','def_rush_dvoa', 'special_teams_dvoa']
    ids = data[['team_id', 'year', 'team', 'week', 'opp']].reset_index(drop=True)
   
    if roll_type == 'mean':
        roll3 = data.groupby(['team','year'])[num_cols].apply(lambda x : x.shift().rolling(roll_value).mean())
        roll2 = data.groupby(['team','year'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-1).mean())
        roll1 = data.groupby(['team','year'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-2).mean())
        roll3 = pd.DataFrame(roll3.combine_first(roll2).combine_first(roll1)).reset_index(drop=True)
        df = pd.concat([ids, roll3], axis=1)
    return df
	

In [10]:
	##Create the current weeks fo team_ids/rows to roll into##
fo_data = pd.read_csv("./current_data/week_4/fo_weekly_update_w"+cur_week_str+".csv")

prev = str(cur_week_int-1)
fo_data_new = fo_data[~fo_data['week'].isnull()]
fo_data_new=fo_data_new.drop_duplicates(subset=['team','year'], keep='last').assign(week=cur_week_str)
fo_data_new['team_id']=fo_data_new['team_id'].str.replace("2022_"+prev, str("2022_"+cur_week_str))

fo_data_new = fo_data_new.sort_values(by=["team","week"], ascending=[True, False])
fo_data_new[fo_data_new.columns[4:]] = np.nan
fo_data_new.head().T

,14,95,50,35,2
team_id,ari_2022_4,atl_2022_4,bal_2022_4,buf_2022_4,car_2022_4
year,2022,2022,2022,2022,2022
team,ari,atl,bal,buf,car
week,4,4,4,4,4
opp,NaN,NaN,NaN,NaN,NaN
total_dvoa,NaN,NaN,NaN,NaN,NaN
off_dvoa,NaN,NaN,NaN,NaN,NaN
off_pass_dvoa,NaN,NaN,NaN,NaN,NaN
off_rush_dvoa,NaN,NaN,NaN,NaN,NaN
def_dvoa,NaN,NaN,NaN,NaN,NaN


In [11]:
fo_data_hist = pd.read_csv("./historic_data/fo_data/fo_weekly_hist.csv")
fo_data_cur = pd.read_csv("./current_data/week_4/fo_weekly_update_w"+cur_week_str+".csv")
fo = pd.concat([fo_data_hist, fo_data_cur, fo_data_new], axis=0)

fo['team'] = fo['team'].map(utils.cleaning_dicts.clean_team_fo).fillna(fo['team'])
fo['opp'] = fo['opp'].map(utils.cleaning_dicts.clean_team_fo).fillna(fo['opp'])

##combine our current season fo data with the new week 4 rows we just made##

fo['team_id'] = fo.apply(lambda df: raiders_fo(df), axis=1) 
fo_roll = rolling_fo(data=fo, roll_value=3, roll_type='mean')
fo_roll.tail().T


,4080,4081,4082,4083,4084
team_id,was_2021_18,was_2022_1,was_2022_2,was_2022_3,was_2022_4
year,2021,2022,2022,2022,2022
team,was,was,was,was,was
week,18,1,2,3,4
opp,nyg,jax,det,phi,NaN
total_dvoa,0.39,NaN,0.5,0.41,0.346667
off_dvoa,0.486667,NaN,0.67,0.625,0.506667
off_pass_dvoa,0.59,NaN,0.58,0.54,0.436667
off_rush_dvoa,0.55,NaN,0.3,0.29,0.35
def_dvoa,0.31,NaN,0.32,0.265,0.253333


In [30]:
fo_data_hist = pd.read_csv("./historic_data/fo_data/fo_weekly_hist.csv")
fo_data_cur = pd.read_csv("./current_data/week_4/fo_weekly_update_w"+cur_week_str+".csv")
fo = pd.concat([fo_data_hist, fo_data_cur, fo_data_new], axis=0)

fo['team'] = fo['team'].map(utils.cleaning_dicts.clean_team_fo).fillna(fo['team'])
fo['opp'] = fo['opp'].map(utils.cleaning_dicts.clean_team_fo).fillna(fo['opp'])


In [32]:
fo.team.value_counts()

ten    136
ari    133
sf     133
pit    133
phi    133
nyj    133
nyg    133
no     133
ne     133
min    133
mia    133
tb     133
atl    133
kc     133
jax    133
ind    133
hou    133
gb     133
det    133
den    133
dal    133
cle    133
cin    133
chi    133
car    133
buf    133
bal    133
sea    133
was    132
lar    101
lac     88
lv      37
Name: team, dtype: int64

In [14]:
#fo_roll.to_csv('fo_roll.csv', index=False)

In [15]:
tgs_new_week = pd.read_csv("./current_data/week_4/team_game_summaries_w"+cur_week_str+".csv")

tgs_new_week = tgs_new_week[~tgs_new_week['week'].isnull()]
tgs_new_week=tgs_new_week.drop_duplicates(subset=['team','year'], keep='last').assign(week=cur_week_str)

tgs_new_week['team_name'] = tgs_new_week['team'].map(utils.cleaning_dicts.clean_team_pff_full).fillna(tgs_new_week['team'])
tgs_new_week['opponent_name'] = tgs_new_week['opponent'].map(utils.cleaning_dicts.clean_team_pff_opp).fillna(tgs_new_week['opponent'])

tgs_new_week['home_or_away']=tgs_new_week['home_or_away'].astype(str)

def home_team(nData):
    if str('@') in nData['home_or_away']:
        return nData['opponent_name']
    else:
        return nData['team_name']

tgs_new_week['home_team'] = tgs_new_week.apply(lambda nData: home_team(nData), axis=1)

def away_team(nData):
    if str('@') in nData['home_or_away']:
        return nData['team_name']
    else:
        return nData['opponent_name']
    
tgs_new_week['away_team'] = tgs_new_week.apply(lambda nData: away_team(nData), axis=1)

def clean_pff_team_summ(df):
##  basic scrubbing to clean data ##
    
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
    df['home_or_away']=np.where(df['home_or_away'] == "@", 1, 0)
    df['wl_int'] = np.where(df['wl'] == "W", 1, 0)
    df=df.replace('-','', regex=True)
    df=df.replace(' ','', regex=True)


    ##  create our unique ids  ##
    df.insert(0, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "team_id_impute", (df['team_name']+'_'+df['year']))
    df.insert(2, "opponent_id", (df['opponent_name']+'_'+df['year']+'_'+df['week']))
    df.insert(3, "home_matchup_id", (df['home_team']+'vs'+df['away_team']+'_'+df['year']+'_'+df['week']))
    df.insert(4, "team_abb", (df['team_name']))

    return df

tgs_new_week = clean_pff_team_summ(tgs_new_week)
tgs_new_week['wl_int'] = ''
tgs_new_week = tgs_new_week.sort_values(by=["team_name","week"], ascending=[True, False])
#tgs_new_week[tgs_new_week.columns[8:]] = np.nan

In [16]:
tgs_data_hist = pd.read_csv("./historic_data/pff_data/team_game_summaries_historic.csv")
tgs_data_cur = pd.read_csv("./current_data/week_4/team_game_summaries_w"+cur_week_str+".csv")
tgs = pd.concat([tgs_data_hist, tgs_data_cur], axis=0)

tgs = tgs[tgs['year'] >= 2014]

tgs['team_name'] = tgs['team'].map(utils.cleaning_dicts.clean_team_pff_full).fillna(tgs['team'])
tgs['opponent_name'] = tgs['opponent'].map(utils.cleaning_dicts.clean_team_pff_opp).fillna(tgs['opponent'])

tgs['home_or_away']=tgs['home_or_away'].astype(str)

def home_team(nData):
    if str('@') in nData['home_or_away']:
        return nData['opponent_name']
    else:
        return nData['team_name']

tgs['home_team'] = tgs.apply(lambda nData: home_team(nData), axis=1)

def away_team(nData):
    if str('@') in nData['home_or_away']:
        return nData['team_name']
    else:
        return nData['opponent_name']
    
tgs['away_team'] = tgs.apply(lambda nData: away_team(nData), axis=1)

def clean_pff_team_summ(df):
##  basic scrubbing to clean data ##

    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
    df['home_or_away']=np.where(df['home_or_away'] == "@", 1, 0)
    df['wl_int'] = np.where(df['wl'] == "W", 1, 0)
    df=df.replace('-','', regex=True)
    df=df.replace(' ','', regex=True)


    ##  create our unique ids  ##
    df.insert(0, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "team_id_impute", (df['team_name']+'_'+df['year']))
    df.insert(2, "opponent_id", (df['opponent_name']+'_'+df['year']+'_'+df['week']))
    df.insert(3, "home_matchup_id", (df['home_team']+'vs'+df['away_team']+'_'+df['year']+'_'+df['week']))
    df.insert(4, "team_abb", (df['team_name']))
    
    return df
   
tgs_clean = clean_pff_team_summ(tgs)
tgs_clean.tail().T


,91,92,93,94,95
unique_team_id,ten_2022_2,ten_2022_3,was_2022_1,was_2022_2,was_2022_3
team_id_impute,ten_2022,ten_2022,was_2022,was_2022,was_2022
opponent_id,buf_2022_2,lv_2022_3,jax_2022_1,det_2022_2,phi_2022_3
home_matchup_id,bufvsten_2022_2,tenvslv_2022_3,wasvsjax_2022_1,detvswas_2022_2,wasvsphi_2022_3
team_abb,ten,ten,was,was,was
team,tennesseetitans,tennesseetitans,washingtoncommanders,washingtoncommanders,washingtoncommanders
year,2022,2022,2022,2022,2022
week,2,3,1,2,3
home_or_away,1,0,0,1,0
team_num,31,31,32,32,32


In [29]:
tgs_clean.opponent_name.value_counts()

lac    133
nyg    133
ten    133
jax    133
hou    133
ind    133
mia    133
car    133
nyj    133
buf    133
tb     133
ne     133
gb     133
min    133
cin    133
cle    133
bal    133
pit    133
chi    133
no     133
kc     133
atl    133
sea    133
det    133
lar    133
dal    133
phi    133
lv     133
was    133
den    133
sf     133
ari    133
Name: opponent_name, dtype: int64

In [19]:
tgs_clean = pd.concat([tgs_clean, tgs_new_week], axis=0)
tgs_clean['year']=tgs_clean['year'].apply(int)
tgs_clean['week']=tgs_clean['week'].apply(int)
tgs_clean = tgs_clean.sort_values(by=["team_abb","year","week"], ascending=[True, True, True])

In [20]:
tgs_clean['special_teams']=tgs_clean['special_teams'].apply(float)

In [21]:

def rolling_tgs(data=None, roll_value=None, roll_type=None):
    
    """
        Args:
        data: input pandas dataframe to be rolled
        roll_value: input the number, default is three ## we will need to modify the function if we want more ##
        roll_type: 'mean','std', or 'var' are the only options at the point
        ## assign mean for a given team & year as opposed to the entire dataset
   
    """
    
    data = data.sort_values(by=["team_abb","year","week"], ascending=[True, True, True])
    num_cols = data.select_dtypes(include=[np.number]).columns.tolist()
    ids = pd.DataFrame(data.select_dtypes(exclude=[np.number])).reset_index(drop=True)
   
    if roll_type == 'mean':
        roll3 = data.groupby(data['team_id_impute'])[num_cols].apply(lambda x : x.shift().rolling(roll_value).mean())
        roll2 = data.groupby(data['team_id_impute'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-1).mean())
        roll1 = data.groupby(data['team_id_impute'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-2).mean())
        roll3 = pd.DataFrame(roll3.combine_first(roll2).combine_first(roll1)).reset_index(drop=True)
        df = pd.concat([ids, roll3], axis=1)
    return df
        
tgs_roll = rolling_tgs(data=tgs_clean, roll_value=3, roll_type='mean')


In [22]:
tgs_roll.sort_values(by=["team_abb","year","week"], ascending=[True, True, True])
tgs_roll.to_csv('tgs_roll.csv', index=False)

In [23]:
fo_roll.columns

Index(['team_id', 'year', 'team', 'week', 'opp', 'total_dvoa', 'off_dvoa',
       'off_pass_dvoa', 'off_rush_dvoa', 'def_dvoa', 'def_pass_dvoa',
       'def_rush_dvoa', 'special_teams_dvoa'],
      dtype='object')

In [24]:
fo_roll_sub = fo_roll[['team_id','total_dvoa', 'off_dvoa',
       'off_pass_dvoa', 'off_rush_dvoa', 'def_dvoa', 'def_pass_dvoa',
       'def_rush_dvoa', 'special_teams_dvoa']]

In [25]:
tgs_roll.columns

Index(['unique_team_id', 'team_id_impute', 'opponent_id', 'home_matchup_id',
       'team_abb', 'team', 'opponent', 'date', 'time', 'wl', 'dateInt', 'Date',
       'output', 'team_name', 'opponent_name', 'home_team', 'away_team',
       'wl_int', 'year', 'week', 'home_or_away', 'team_num', 'game_id', 'pf',
       'pa', 'overall_performance', 'offense', 'pass', 'pass_blocking',
       'receiving', 'rushing', 'run_blocking', 'defense', 'rush_defense',
       'tackling', 'pass_rush', 'coverage', 'special_teams'],
      dtype='object')

In [26]:
## merge tgs_roll, with fo_roll by unique_team_ids, then bring on the spreads data ##
tgs_roll_sub = tgs_roll[['unique_team_id','home_matchup_id','pf', 'pa', 'wl_int', 'overall_performance', 'offense', 'pass',
       'pass_blocking', 'receiving', 'rushing', 'run_blocking', 'defense',
       'rush_defense', 'tackling', 'pass_rush', 'coverage']]

#mrg = pd.merge(fo_roll, tgs_roll_sub, left_on="team_id", right_on="unique_team_id", how='left')

In [27]:
mrg = pd.merge(fo_roll, tgs_roll_sub, left_on="team_id", right_on="unique_team_id", how='left')

In [28]:
spread_comb.head()

,home_matchup_id,away_matchup_id,team_id,schedule_season,schedule_week,team_home_abb,score_home,score_away,away_team_abb,team_favorite_id,...,over_under_result,precip,dome,temperature,wind_mph,team_abb,homeoraway,fav_homeoraway,ats_w,ats_l
0,arivslac_2014_1,lac@ari_2014_1,ari_2014_1,2014,1,ari,18.0,17.0,lac,ari,...,under,0.0,1.0,70.0,0.0,ari,1,0,0,1
1,arivssf_2014_3,sf@ari_2014_3,ari_2014_3,2014,3,ari,23.0,14.0,sf,sf,...,under,0.0,1.0,70.0,0.0,ari,1,1,1,0
2,arivswas_2014_6,was@ari_2014_6,ari_2014_6,2014,6,ari,30.0,20.0,was,ari,...,over,0.0,1.0,70.0,0.0,ari,1,0,1,0
3,arivsphi_2014_8,phi@ari_2014_8,ari_2014_8,2014,8,ari,24.0,20.0,phi,ari,...,under,0.0,1.0,70.0,0.0,ari,1,0,1,0
4,arivslar_2014_10,lar@ari_2014_10,ari_2014_10,2014,10,ari,31.0,14.0,lar,ari,...,over,0.0,1.0,70.0,0.0,ari,1,0,1,0


In [20]:
spreads_for_tgs = spread_comb[['team_id','home_matchup_id','fav_homeoraway','team_favorite_id','fav_cover','spread_favorite',
       'over_under_line', 'starting_spread', 'Total Score Open', 'spread_movement','precip', 'dome', 'temperature', 'wind_mph',
       'ou_movement', 'strong_movement', 'fav_team_stronger', 'fav_spread','ats_w','ats_l']]

tgs_model = pd.merge(spreads_for_tgs, mrg, on='team_id', how='left')

In [21]:
def fav_ids(nData):
    if str(nData['team_favorite_id']) in str(nData['team_id']):
        return nData['team_id']
    else:
        pass
tgs_model['fav_team_id'] = tgs_model.apply(lambda nData: fav_ids(nData), axis=1)


favs = tgs_model[~tgs_model['fav_team_id'].isnull()]
not_fav = tgs_model[tgs_model['fav_team_id'].isnull()]

In [22]:
favs.shape

(2132, 48)

In [23]:
not_fav.shape

(2130, 48)

In [24]:
not_fav_df = tgs_model[tgs_model.team_id.isin(not_fav.team_id)]

favs = favs[['team_id','week','year','spread_favorite','over_under_line','fav_cover','starting_spread',
       'Total Score Open', 'spread_movement', 'ou_movement', 'strong_movement',
       'fav_team_stronger']]

def id_prefix(prefix=None, df=None):
    id = df[['home_matchup_id']]
    temp = df.drop(['team_id', 'year', 'team', 'week', 'opp','team_favorite_id', 'home_matchup_id',
       'spread_favorite', 'over_under_line', 'starting_spread',
       'Total Score Open', 'spread_movement', 'ou_movement', 'strong_movement',
       'fav_team_stronger', 'fav_spread', 'fav_cover', 'ats_w', 'ats_l',
       'fav_team_id'], axis=1)
    temp = temp.add_prefix(prefix)
    id = pd.concat([id, temp], axis=1)
    return id
    

not_fav_df = id_prefix(prefix="dawg", df=not_fav_df)

fav_df = tgs_model[['team_id','total_dvoa', 'off_dvoa',
       'off_pass_dvoa', 'off_rush_dvoa', 'def_dvoa', 'def_pass_dvoa',
       'def_rush_dvoa', 'special_teams_dvoa', 'pa','pf', 'overall_performance',
       'offense', 'pass', 'pass_blocking', 'receiving', 'rushing',
       'run_blocking', 'defense', 'rush_defense', 'tackling', 'pass_rush',
       'coverage', 'team_favorite_id', 'home_matchup_id']]

In [25]:
fin_df = pd.merge(favs, fav_df, on='team_id', how='left').merge(not_fav_df, on='home_matchup_id', how='left')
fin_df = fin_df[~fin_df['dawgtotal_dvoa'].isnull()]

In [26]:
fin_df.drop_duplicates('home_matchup_id', inplace=True)

In [27]:
fin_df

,team_id,week,year,spread_favorite,over_under_line,fav_cover,starting_spread,Total Score Open,spread_movement,ou_movement,...,dawgpass,dawgpass_blocking,dawgreceiving,dawgrushing,dawgrun_blocking,dawgdefense,dawgrush_defense,dawgtackling,dawgpass_rush,dawgcoverage
1,ari_2014_6,6,2014.0,-5.0,47.0,cover,-3.5,45.0,-1.5,2.0,...,62.100000,73.700000,67.000000,50.633333,61.100000,44.300000,56.433333,57.766667,61.000000,36.000000
2,ari_2014_8,8,2014.0,-1.0,49.0,cover,-3.0,48.0,2.0,1.0,...,59.766667,66.133333,63.533333,53.500000,75.433333,69.866667,69.300000,73.133333,67.866667,65.666667
5,ari_2015_3,3,2015.0,-7.0,45.0,cover,-6.0,44.0,-1.0,1.0,...,68.500000,70.900000,65.200000,61.900000,66.600000,59.950000,68.350000,66.400000,53.950000,57.200000
7,ari_2015_7,7,2015.0,-10.0,49.5,no cover,-7.5,48.5,-2.5,1.0,...,60.366667,85.600000,65.400000,64.500000,70.533333,62.433333,65.066667,56.766667,59.066667,60.600000
8,ari_2015_11,11,2015.0,-4.0,48.0,no cover,-3.5,47.0,-0.5,1.0,...,63.100000,75.900000,64.200000,64.300000,68.133333,66.933333,61.533333,67.800000,62.733333,67.700000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2137,tb_2021_10,10,2021.0,-10.0,50.5,no cover,-9.5,51.5,-0.5,-1.0,...,60.866667,70.700000,60.600000,55.466667,75.433333,61.133333,64.666667,60.600000,65.833333,55.933333
2138,sea_2021_12,12,2021.0,-1.5,46.5,no cover,-2.5,46.5,1.0,0.0,...,72.500000,75.466667,69.266667,59.966667,78.466667,62.066667,60.400000,60.900000,59.100000,65.866667
2139,dal_2021_14,14,2021.0,-6.5,48.0,cover,-4.5,49.0,-2.0,-1.0,...,74.900000,72.933333,71.333333,60.133333,73.233333,68.833333,64.566667,58.433333,66.533333,69.166667
2140,phi_2021_17,17,2021.0,-6.0,44.5,no cover,-3.0,45.0,-3.0,-0.5,...,48.066667,67.433333,60.200000,61.866667,65.000000,49.933333,57.333333,68.300000,55.333333,49.933333


In [28]:
fin_df.sort_values(by=["team_favorite_id","year","week"], ascending=[True, True, True])
fin_df.to_csv('mod.csv', index=False)